In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


In [8]:
import torch
import os
import time
import cv2
import numpy as np
from models import create_model
from util import util
from PIL import Image
import sys


In [13]:
capture_delay = 0.25  # Delay between capturing images in seconds
images_size = (256, 256)
brightness_factor = 1.8  # Factor by which to increase brightness

def brighten_image(image, factor=1.2):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype=np.float64)
    hsv[:, :, 2] = hsv[:, :, 2] * factor
    hsv[:, :, 2][hsv[:, :, 2] > 255] = 255
    hsv = np.array(hsv, dtype=np.uint8)
    brightened_img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return brightened_img

def capture_images_auto(num_images, output_dir='data/me/train/hat', interval=0.25):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    date_stamp = time.strftime("%Y%m%d-%H%M%S")
    img_counter = 0

    while img_counter < num_images:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break

        # Display the live feed
        cv2.imshow('Webcam Feed', frame)

        # Brighten the frame
        brightened_frame = brighten_image(frame, brightness_factor)

        img_name = os.path.join(output_dir, f'{date_stamp}_{img_counter}.jpg')
        frame_resized = cv2.resize(brightened_frame, images_size)
        cv2.imwrite(img_name, frame_resized)
        print(f'Captured {img_name}')

        img_counter += 1
        time.sleep(interval)

        # Break the loop if the user presses 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def preprocess_images(image_paths):
    processed_images = []
    for image_path in image_paths:
        img = cv2.imread(image_path)
        img = cv2.resize(img, images_size)  # Resize to output size 256x256
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = (img / 127.5) - 1.0  # Normalize to [-1, 1]
        processed_images.append(img)
    return np.array(processed_images)

# Example usage
if __name__ == '__main__':
    num_images = 400
    output_dir = 'data/with_hat_class'
    capture_images_auto(num_images, output_dir, capture_delay)

    image_paths = [os.path.join(output_dir, fname) for fname in os.listdir(output_dir) if fname.endswith('.jpg')]
    processed_images = preprocess_images(image_paths)
    

Captured data/with_hat_class\20240623-180745_0.jpg
Captured data/with_hat_class\20240623-180745_1.jpg
Captured data/with_hat_class\20240623-180745_2.jpg
Captured data/with_hat_class\20240623-180745_3.jpg
Captured data/with_hat_class\20240623-180745_4.jpg
Captured data/with_hat_class\20240623-180745_5.jpg
Captured data/with_hat_class\20240623-180745_6.jpg
Captured data/with_hat_class\20240623-180745_7.jpg
Captured data/with_hat_class\20240623-180745_8.jpg
Captured data/with_hat_class\20240623-180745_9.jpg
Captured data/with_hat_class\20240623-180745_10.jpg
Captured data/with_hat_class\20240623-180745_11.jpg
Captured data/with_hat_class\20240623-180745_12.jpg
Captured data/with_hat_class\20240623-180745_13.jpg
Captured data/with_hat_class\20240623-180745_14.jpg
Captured data/with_hat_class\20240623-180745_15.jpg
Captured data/with_hat_class\20240623-180745_16.jpg
Captured data/with_hat_class\20240623-180745_17.jpg
Captured data/with_hat_class\20240623-180745_18.jpg
Captured data/with_hat

In [3]:
!pip install torch torchvision opencv-python pillow


In [4]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix


Cloning into 'pytorch-CycleGAN-and-pix2pix'...


In [5]:
cd pytorch-CycleGAN-and-pix2pix


c:\Users\agran\OneDrive - Holon Institute of Technology\3. Year 3\ראייה ממוחשבת\ComputerVision\GAN_Project\To_Submit\pytorch-CycleGAN-and-pix2pix


In [9]:
sys.path.append('/GAN_Project\pytorch-CycleGAN-and-pix2pix')  # Adjust this path as necessary


In [10]:

checkpoints_dir = 'C:/Users/agran/OneDrive - Holon Institute of Technology/3. Year 3/ראייה ממוחשבת/ComputerVision/GAN_Project/saved_models'  # Adjust this path
print(os.listdir(checkpoints_dir))


['drive-download-20240622T153818Z-001', 'latest_net_G_A.pth', 'latest_net_G_B.pth', 'sesion_1', 'sesion_2', 'sesion_3', 'sesion_4']


In [15]:

class CustomTestOptions:
    def __init__(self):
        self.dataroot = 'C:/Users/agran/OneDrive - Holon Institute of Technology/3. Year 3/ראייה ממוחשבת/ComputerVision/GAN_Project/dataset'
        self.name = ''
        self.model = 'cycle_gan'
        self.checkpoints_dir = 'C:/Users/agran/OneDrive - Holon Institute of Technology/3. Year 3/ראייה ממוחשבת/ComputerVision/GAN_Project/saved_models'
        self.no_dropout = True
        self.num_threads = 0
        self.batch_size = 1
        self.serial_batches = True
        self.no_flip = True
        self.display_id = -1
        self.phase = 'test'
        self.epoch = 'latest'
        self.eval = True
        self.load_size = 256
        self.crop_size = 256
        self.gpu_ids = [0] if torch.cuda.is_available() else []

        self.isTrain = False
        self.input_nc = 3
        self.output_nc = 3
        self.ngf = 64
        self.ndf = 64
        self.netD = 'basic'
        self.netG = 'resnet_9blocks'
        self.preprocess = 'resize_and_crop'
        self.display_winsize = 256
        self.init_type = 'normal'
        self.init_gain = 0.02
        self.suffix = ''
        self.verbose = False
        self.norm = 'instance'
        self.dataset_mode = 'unaligned'
        self.direction = 'AtoB'
        self.lambda_identity = 0.5
        self.lambda_A = 10.0
        self.lambda_B = 10.0
        self.pool_size = 50
        self.lr = 0.0002
        self.beta1 = 0.5
        self.load_iter = 0
        self.continue_train = False
        self.epoch_count = 1
        self.niter = 100
        self.niter_decay = 100

# Create a new CustomTestOptions object
opt = CustomTestOptions()

# Initialize the model
model = create_model(opt)
model.setup(opt)
model.eval()

def preprocess_frame(frame, opt):
    """ Preprocess the input frame """
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    image = image.resize((opt.load_size, opt.load_size), Image.BICUBIC)
    image = np.array(image).astype(np.float32)
    image = (image / 255.0 - 0.5) / 0.5
    image = np.transpose(image, (2, 0, 1))
    image = torch.tensor(image).unsqueeze(0)
    return image

def postprocess_frame(tensor):
    """ Postprocess the output tensor to a frame """
    image = util.tensor2im(tensor)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image

# Create a dummy tensor for 'B' input
dummy_B = torch.zeros((1, 3, opt.load_size, opt.load_size))

# Set up the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    frame_tensor = preprocess_frame(frame, opt)
    
    # Run the model on the frame
    data = {'A': frame_tensor, 'B': dummy_B, 'A_paths': '', 'B_paths': ''}
    model.set_input(data)
    model.test()
    visuals = model.get_current_visuals()
    
    # Postprocess the output
    fake_B = visuals['fake_B']
    output_frame = postprocess_frame(fake_B)
    
    # Resize output frame to match original frame dimensions
    output_frame_resized = cv2.resize(output_frame, (frame.shape[1], frame.shape[0]))

    # Display the original and generated frames side by side
    combined_frame = np.hstack((frame, output_frame_resized))
    cv2.imshow('CycleGAN Live Inference', combined_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


initialize network with normal
initialize network with normal
model [CycleGANModel] was created
loading the model from C:/Users/agran/OneDrive - Holon Institute of Technology/3. Year 3/ראייה ממוחשבת/ComputerVision/GAN_Project/saved_models\latest_net_G_A.pth
loading the model from C:/Users/agran/OneDrive - Holon Institute of Technology/3. Year 3/ראייה ממוחשבת/ComputerVision/GAN_Project/saved_models\latest_net_G_B.pth
---------- Networks initialized -------------
[Network G_A] Total number of parameters : 11.378 M
[Network G_B] Total number of parameters : 11.378 M
-----------------------------------------------
